In [1]:
# Welcome to the Kalshi REST v2 Starter Code!

# pypi client: recommended for more advanced programmers
import kalshi_python

# starter client: recommended for all levels of programming experience (what this client is implemented using)
from KalshiClientsBaseV2 import ExchangeClient
import time
import json
import uuid

In [2]:
# To start off, you need to have created an account at https://kalshi.com (Production) 
# or an account on the Demo https://demo.kalshi.co/

prod_email = "" # change these to be your personal credentials
prod_password = "" # (for extra security, we recommend using a config file)

demo_email = "ketanss@bu.edu" # change these to be your personal credentials
demo_password = "Ketankalshi2001#" # (for extra security, we recommend using a config file)

# for prod
prod_api_base = "https://trading-api.kalshi.com/trade-api/v2"

# for demo
demo_api_base = "https://demo-api.kalshi.co/trade-api/v2"

## if wanting to test in prod:
# exchange_client = ExchangeClient(exchange_api_base=prod_api_base, email = prod_email, password = prod_password)

## if wanting to test in demo
exchange_client = ExchangeClient(exchange_api_base = demo_api_base, email = demo_email, password = demo_password)

# first we will check on the exchange status to confirm you are properly connected...
print(exchange_client.get_exchange_status())

{'exchange_active': True, 'trading_active': True}


In [6]:
# Next, let's look at event level data by passing an event ticker to the get_event endpoint...

event_ticker = "HIGHNY-23OCT09"
event_params = {'event_ticker': event_ticker}
event_response = exchange_client.get_event(**event_params)

print('keys:', event_response.keys())
print()
print('event object:', event_response['event'])
print()
print(event_response['markets'])

keys: dict_keys(['event', 'markets'])

event object: {'event_ticker': 'HIGHNY-23OCT09', 'series_ticker': 'HIGHNY', 'sub_title': 'On Oct 9, 2023', 'title': 'What will the high temp be in NYC on Oct 9, 2023?', 'mutually_exclusive': True, 'category': 'Climate and Weather', 'strike_date': '2023-10-10T03:59:00Z'}

[{'ticker': 'HIGHNY-23OCT09-T56', 'event_ticker': '', 'market_type': 'binary', 'title': '', 'subtitle': '55° or below', 'yes_sub_title': '55° or below', 'no_sub_title': 'Outside 55° or below', 'open_time': '2023-10-08T14:00:00Z', 'close_time': '2023-10-10T03:59:00Z', 'expected_expiration_time': '2023-10-10T14:00:00Z', 'expiration_time': '2023-10-16T14:00:00Z', 'latest_expiration_time': '2023-10-16T14:00:00Z', 'settlement_timer_seconds': 3600, 'status': 'active', 'response_price_units': 'usd_cent', 'notional_value': 100, 'tick_size': 1, 'yes_bid': 0, 'yes_ask': 1, 'no_bid': 0, 'no_ask': 0, 'last_price': 0, 'previous_yes_bid': 0, 'previous_yes_ask': 0, 'previous_price': 0, 'volume':

In [8]:
tickers = []
for i in event_response['markets']:
    tickers.append(i['ticker'])
tickers

['HIGHNY-23OCT09-T56',
 'HIGHNY-23OCT09-B56.5',
 'HIGHNY-23OCT09-B58.5',
 'HIGHNY-23OCT09-B60.5',
 'HIGHNY-23OCT09-B62.5',
 'HIGHNY-23OCT09-T63']

In [9]:
tickers_ranges = [float(x.split('-')[-1][1:]) for x in tickers]
tickers_ranges

[56.0, 56.5, 58.5, 60.5, 62.5, 63.0]

In [10]:

for i in range(1,len(tickers_ranges)-1):
    tickers_ranges[i] = [tickers_ranges[i]-0.5,tickers_ranges[i]+0.5]
    
tickers_ranges
tickers_ranges[0] = [-20,tickers_ranges[0]-1]
tickers_ranges[-1] = [tickers_ranges[-1]+1,150]


In [11]:
tickers_ranges

[[-20, 55.0],
 [56.0, 57.0],
 [58.0, 59.0],
 [60.0, 61.0],
 [62.0, 63.0],
 [64.0, 150]]

In [12]:
pred = round(62.51)
def get_ticker(tickers_ranges,pred):
    for i in range(len(tickers_ranges)):
        if pred>=tickers_ranges[i][0] and pred<=tickers_ranges[i][1]:
            return i
get_ticker(tickers_ranges,pred)

4

In [47]:
# Now let's suppose we wanted to place a trade on one of these markets... 
# to do so, we would first want to check out available balance...

current_balance = exchange_client.get_balance()
current_balance['balance']

998236

In [13]:
# Now onto placing an order...
# There are many different ways to think about placing orders at Kalshi. 
# The following param examples will walk through some of those

# Limit buy order for 10 units at 30c No on GDPW-22-A3

ticker = tickers[3]

order_params = {'ticker':ticker,
                    'client_order_id':str(uuid.uuid4()),
                    'type':'limit',
                    'action':'buy',
                    'side':'yes',
                    'count':1,
                    'yes_price':None, # yes_price = 100 - no_price
                    'no_price':50, # no_price = 100 - yes_price
                    'expiration_ts':None,
                    'sell_position_floor':None,
                    'buy_max_cost':None}

exchange_client.create_order(**order_params)


{'ticker': 'HIGHNY-23OCT09-B60.5', 'client_order_id': '9caba753-7d52-4ee4-8fca-1170eea57823', 'side': 'yes', 'action': 'buy', 'count': 1, 'type': 'limit', 'no_price': 50}


{'order': {'order_id': '8863b5d0-8dba-4181-bfa1-e06f00f01307',
  'user_id': '3c449eea-e6ea-4ff6-85e5-753d49e743f3',
  'ticker': 'HIGHNY-23OCT09-B60.5',
  'status': 'resting',
  'yes_price': 50,
  'no_price': 50,
  'created_time': '2023-10-09T14:05:25.765531Z',
  'expiration_time': None,
  'action': 'buy',
  'side': 'yes',
  'type': 'limit',
  'client_order_id': '9caba753-7d52-4ee4-8fca-1170eea57823',
  'order_group_id': ''}}